# Monadic functions

## Outline

* What are they

* Often used monadic functions
  - Lifting functions

  - `sequence` and `sequence_`

  - Mapping functions

  - `filterM` and `foldM`

  - `when` and `unless`

* Handling exceptions

In this lesson, we will learn about monadic functions and how you can use them.

## What are they

For the start let's write out some short definitions that we already know:
- A **monad** is a typeclass that defines `>>=` and `return` (minimally).

- A **monadic value** is a value contextualized by a type constructor that has an instance of the monad type class.

- The **inner value** of a monadic value is the actual value that was contextualized by a type constructor.

That brings us to the definition of a **monadic function**:
- A **monadic function** is a function that produces a monadic value. 

- **Monadic code** is code defined inside of a monadic function.

This is a very broad definition. We can say all functions that we wrote so far which returned `IO a`, `Maybe a` and `[a]`, can be called monadic functions. 

Further we distinguish between two types of monadic functions:
- **Classic monadic functions** that are in the form `f :: a -> m b`, where a is the type of the inner value of the monad.

- **Loose monadic functions** that are in the form `f :: anything -> m b`, where the input of the function really doesn't matter.

We will be focusing in this chapter on monadic functions which are available from Haskell modules and make writing monadic code easier.

## Often used monadic functions

Here we cover some of the often used available monadic functions, explain what they do and when to use them.

All of the functions presented in this chapter are contained in the **Control.Monad** module.

### Lifting functions

The first of the lifting functions is the `liftM` function. Its definition is:
```haskell
liftM :: Monad m => (a1 -> r) -> m a1 -> m r 
liftM f m1 = do 
    x1 <- m1
    return (f x1) 
```

It promotes a function to a monad. Let's look at an example.

In [ ]:
import Control.Monad (liftM)

add1 :: Num a => a -> a
add1 x1 = x1 + 1

var1 :: Maybe Int
var1 = Just 2

result1 :: Maybe Int
result1 = liftM add1 var1

result1

We notice that we could get the same result if we used the Functor operator `fmap` instead of `liftM`.

The reason why `liftM` is usefull is because it is expressed with the monad operators `>>=` and `return`. 

If we define an instance of monad for a type we created we also have to define an instance of Applicative and Functor.

And in the functor definition we can simply say that `fmap = liftM`. 

In case we have a function that takes in two arguments we can use the `liftM2` function.
```haskell
liftM2 :: Monad m => (a1 -> a2 -> r) -> m a1 -> m a2 -> m r 
liftM2 f m1 m2 = do 
    x1 <- m1 
    x2 <- m2 
    return (f x1 x2)
```

In [ ]:
import Control.Monad (liftM2)

liftM2 (+) [0,1] [0,2]
liftM2 (+) (Just 1) Nothing

Also the functions `liftM3`, `liftM4` and `liftM5` exist. The numbers represent the number of arguments that their input functions accept.

The last lifting function we present is `ap`, which is defined as:
```haskell
ap :: Monad m => m (a -> b) -> m a -> m b 
ap m1 m2 = do 
    f1 <- m1 
    x1 <- m2 
    return (f1 x1)
```

It works the same as the `<*>` operator from Applicative type class. It is useful for the same reason as the `liftM` function.

If we define an instance of monad for our type, we also have to define an instance of applicative and can say `(<*>) = ap`. 

### The `sequence` and `sequence_` functions

The definition of the `sequence` function is:
```haskell
sequence :: (Traversable t, Monad m) => t (m a) -> m (t a)
```

It evaluates each monadic action in the traversable structure from left to right, and collects the results.

Here is an example of this function:

In [ ]:
import Control.Monad (sequence)

var1 :: Maybe [Int]
var1 = Just [1,2,3,4]

list1 :: [Maybe Int]
list1 = sequence var1

list1

We could also define a function that works reversed as the `sequence` function for lists.

In [ ]:
import Control.Applicative (liftA2)

sequenceR :: (Applicative f) => [f a] -> f [a]
sequenceR = foldr (liftA2 (:)) (pure [])

list2Var :: Maybe [Int]
list2Var = sequenceR list1

list2Var

The `liftA2` function from the **Control.Applicative** module basically works the same as the `liftM2` function.
```haskell
liftA2 :: (a -> b -> c) -> f a -> f b -> f c 
```
The difference is that it promotes a function to an applicative instead of a monad.

The function `sequence_` works similar as `sequence` just that it throws away the result.
```haskell
sequence_ :: (Foldable t, Monad m) => t (m a) -> m () 
```

This comes useful when you need to get the monad context out of the variable.

For instance in the code example below we use it to be able to print elements of a list one by one.

In [ ]:
import Control.Monad (sequence_)

mapPrint :: Show a => [a] -> IO ()
mapPrint xs = sequence_ . map print $ xs

mapPrint [1..3]

If we would not use the `sequence_` function we would get a compile error.

In Haskell it is a general convention if a function has an underscore at the end it means that it throws away the result. 

### Mapping functions

The first mapping functions we introduce are `mapM` and `mapM_`. Their type signatures are:
```haskell
mapM :: (Traversable t, Monad m) => (a -> m b) -> t a -> m (t b) 

mapM_ :: (Foldable t, Monad m) => (a -> m b) -> t a -> m () 
```

The `mapM` function takes a monadic function and applies it over each element of a traversable type.

The result is a traversable type in a monad context. Here is an example of how this works:

In [ ]:
import Control.Monad (mapM)
import Text.Read (readMaybe)

list1, list2 :: [String]
list1 = ["a", "2", "3"]
list2 = ["1", "2", "3"]

maybeProduct :: [String] -> Maybe Int
maybeProduct list =
    let parsedList = mapM readMaybe list
    in product <$> parsedList

print $ maybeProduct list1
print $ maybeProduct list2

The `mapM_` function takes a monadic function, applies it over a traversable element and throws away the result.

This is usefull when your input monadic function produces side effects and does not return a meaningful result.

We can rewrtite the print statements from the previous code example to one line:

In [ ]:
import Control.Monad (mapM_)

mapM_ print [list1, list2]

The next pair of mapping functions we talk about are `forM` and `forM_`. Their type signatures are:
```haskell
forM :: (Traversable t, Monad m) => t a -> (a -> m b) -> m (t b) 

forM_ :: (Foldable t, Monad m) => t a -> (a -> m b) -> m () 
```

They work the same as the `mapM` and `mapM_` functions just that their input arguments are flipped.

This can come useful when simulating a for loop in monadic code as in the example below.

In [ ]:
import Control.Monad (forM_)

list :: [Int]
list = [4,2,8,5,1]

printNums :: [Int] -> IO ()
printNums list = do
    forM_ [1..length list] $ \ind -> do
        let indString = show ind
            element = show (list !! (ind - 1))
        putStrLn $ "Element with index " ++ indString ++ " is: " ++ element

printNums list

### The `filterM` and `foldM` functions

The `filterM` function has the following type signature:
```haskell
filterM :: Applicative m => (a -> m Bool) -> [a] -> m [a] 
```

It takes in a monadic function that where the return result is parameterized by a bool and applies it to a list.

The returned result is a monadic value parameterized by a list. Below is a code example where we filter out numbers from a list. 

In [ ]:
import Control.Monad (filterM)

nums1, nums2 :: [Int]
nums1 = [1,2,3,4,5]
nums2 = [1,2,4,5,7]

divisibleBy3 :: Int -> Bool
divisibleBy3 n = n `mod` 3 == 0

filterFunc :: Int -> Maybe Bool
filterFunc n = if divisibleBy3 n 
               then Nothing 
               else Just (even n)

filterM filterFunc nums1
filterM filterFunc nums2

So we see this comes useful when we filter out lists and want to work in a monadic context. 

We can also use `filterM` function to compure all possible combinations from elements of a list:

In [ ]:
filterM (const [True,False]) [1,2,3]

This works because our filter function returns the type `[Bool]` so the return type of `filterM` becomes `[[a]]`.

If you are courious how this works you can look at the following implementation of the `filterM` function and try to understand it with the example above.
```haskell
filterM :: Monad m => (a -> m Bool) -> [a] -> m [a]
filterM p []     = return []
filterM p (x:xs) =
    let rest = filterM p xs in
        do b <- p x
           if b then liftM (x:) rest
           else                 rest
```

Now we can also compute for instance all possible sums from elements of a list. 

The number 3 will get repeated because the combinations `[1,2]` and `[3]` give the same result.

In [ ]:
map sum $ filterM (const [True,False]) [1,2,3]

The next function is `foldM` that has the type signature:
```haskell
foldM :: (Foldable t, Monad m) => (b -> a -> m b) -> b -> t a -> m b 
```

The `foldM` function is analogous to `foldl`, except that its result is encapsulated in a monad. 

Note that `foldM` works from left-to-right over the list arguments. 

From hackage we get the following explanation how `foldM` works:
```haskell
foldM f a1 [x1, x2, ..., xm] ==
do
  a2 <- f a1 x1
  a3 <- f a2 x2
  ...
  f am xm
```

The `foldM` function is useful when we are doing iterations inside a monadic context as `IO`.

Let's look at an example where we want to sum the numbers of a list and print out the workflow.

In [ ]:
import Control.Monad (foldM)

sumAndPrint :: (Foldable t, Show b, Num b) => t b -> IO b
sumAndPrint list = foldM (\a b ->
                          putStrLn (show a ++ "+" ++ show b ++
                          "=" ++ show (a+b)) >>
                          return (a+b)) 0 list

main :: IO ()
main = do
    result <- sumAndPrint [1..5]
    putStrLn $ "Final result is: " ++ show result

main

Another example of how to use the `foldM` is when reading iterativly input data from the user.

We define the functions `f1` and `f2` that work the same. The first uses recursion and the second `foldM`. 

In [ ]:
import Control.Monad (foldM)

f1 :: Int -> IO [String]
f1 0 = return []
f1 n = do 
         v <- getLine
         vs <- f1 (n-1)
         return $ v : vs

f2 :: Int -> IO [String]
f2 n = do foldM fn [] [1..n]
  where fn acc _ = do x <- getLine
                      return (acc ++ [x])

main :: IO ()
main = do
    result1 <- f1 3
    print result1

    result2 <- f2 3
    print result2

main

### The `when` and `unless` functions

The `when` function has the following definition:
```haskell
when :: Applicative f => Bool -> f () -> f () 
when p s  = if p then s else pure ()
```

It can be used when you want to execute code only if a condition is true and do nothings otherwise.

Let's say we want to print out elements of a list which are smaller then their left-side neighbour.

In [ ]:
import Control.Monad (when, forM_)

list1 :: [Int]
list1 = [4,5,8,2,1]

largerNeighbours :: [Int] -> IO ()
largerNeighbours list = do
    forM_ [1..(length list - 1)] $ \ind -> do
        let elemet = list !! ind
            leftNeighbour = list !! (ind -1)
            test = leftNeighbour < elemet
        when test $ do
            putStrLn $ "Element " ++ show (ind + 1) ++ " is larger then it's left-side neighbour:"
            putStrLn $ show leftNeighbour ++ " < " ++ show elemet

largerNeighbours list1

The `unless` function does the opposite of the `when` function and performs some code only if the input condition is not true.
```haskell
unless :: (Applicative f) => Bool -> f () -> f ()
unless p s =  if p then pure () else s
```

## Handeling exceptions

Handeling exceptions is a wide area to talk about. It is often used in `IO` functions because of unpredictable input data.

The main benifit of handeling exceptions is that you can interrupt the code flow in a function by earlier returning a result.

In this chapter we will showcase only handeling exceptions in `IO` function and we will use monadic functions to do this. 

The functions and data types we will need are defined in the modules **Control.Exception** and **Data.Typeable**. 

To be able to work with exceptions we need to define out exception type and make an `Exception`instance of it. 

In [ ]:
import Control.Exception (Exception)
import Data.Typeable (Typeable)

data MyException = MyException deriving (Show, Typeable)
instance Exception MyException

For the `MyException` type we derive the `Typeable` type class that is defined in the **Data.Typeable** module. 

It allows a concrete representation of a type to be calculated.

The `Exception` type class is contained in the **Control.Exception** module and is defined as:
```haskell
class (Typeable e, Show e) => Exception e where
    toException   :: e -> SomeException
    fromException :: SomeException -> Maybe e

    toException = SomeException
    fromException (SomeException e) = cast e

    displayException :: e -> String
    displayException = show
```

It does not have a minimal complete definition so we do not have to define any functions when making an instance of it.

But it does require that the type for which we are making an instance, implements the `Show` and `Typeable` type classes.

Once we have our execption type we can look at the monadic functions which we need to work with `IO` exceptions.

They are both contained in the **Control.Exception** module.

The `handle` function takes in a `IO` exception handle function and a `IO` action which it performs. 
```haskell
handle :: Exception e => (e -> IO a) -> IO a -> IO a 
```

The `throwIO` function takes in an exception and returns a `IO` result.
```haskell
throwIO :: Exception e => e -> IO a 
```

Below is a game where the computer generates a random number from 1 to 10 and the user tries to guess which one it is.

He has 2 attampts in the first try and 4 attemps in the second try. If he succeds in any of the cases we interupt the function and return a message.

In [ ]:
import Control.Exception (handle, throwIO)
import System.Random (mkStdGen, Random(randomR))
import Text.Read (readMaybe)

main :: IO ()
main = do
    putStrLn "Starting game. Generating random number from 1 to 10."
    let (number, _) = randomR (1, 10) (mkStdGen 1)

    msg <- playGame number 
    putStrLn msg

playGame :: Int -> IO String
playGame n = handle (\ MyException -> return "Game successfully finished.") $ do
    putStrLn "Input 2 numbers:"
    input1 <- getLine
    let nums1 = take 2 $ map readMaybe (words input1) :: [Maybe Int]
        test1 = Just n `elem` nums1
    when test1 $ do
        putStrLn "Guessed number correctly with 2 attempts!"
        throwIO MyException -- stops executing the function

    putStrLn "Input 4 numbers:"
    input2 <- getLine
    let nums2 = take 4 $ map readMaybe (words input2) :: [Maybe Int]
        test2 = Just n `elem` nums2
    when test2 $ do
        putStrLn "Guessed number correctly with 4 attempts!"
        throwIO MyException -- stops executing the function

    return "Game finished without success."

## Recap

In this lesson we've discussed:

- the definition of monadic functions

- monadic functions from **Control.Monad** module

- how to handle `IO` exceptions with monadic functions